In [6]:
search_for = 114
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.006775379180908203
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 125
all cases: 515991311
type: [1, 1, 1, 1, 114] 114
cases of this type: 168896016
10000 35.38005489475617
20000 37.2234987551504
30000 46.304509618063385
40000 44.97909689361439
50000 189.4005451755419
60000 163.53955724631763
70000 174.74445279622407
80000 158.12468704096275
90000 203.03289530553917
100000 200.06412541509522
110000 218.65138404022633
120000 190.40195180428674
130000 213.21660930112037
140000 194.04660189741804
150000 193.48532349383564
160000 199.84678202485622
170000 183.86111479093105
180000 229.8322348750263
190000 208.27322020633204
200000 247.69634695441468
210000 221.2673268214163
220000 270.52416657387704
230000 315.39119012558876
240000 379.73141021731976
250000 356.78946917753973
260000 364.75342499588413
270000 364.1279067504912
280000 357.4340767902769
290000 373.73958411743655
300000 372.2421542779004
310000 366

3080000 375.7472839361951
3090000 381.26921943252256
3100000 379.0832121592765
3110000 393.97568167877694
3120000 380.03106729396114
3130000 407.2141106443383
3140000 371.86086766007475
3150000 406.78474434897566
3160000 355.85303997393606
3170000 406.93290719101134
3180000 363.1389799719057
3190000 397.26593224453717
3200000 359.52383878289635
3210000 379.0101148102512
3220000 365.2438003686521
3230000 361.3171515402987
3240000 375.38849645789895
3250000 365.9612554021677
3260000 386.9043723553159
3270000 396.18419091128504
3280000 375.13190370965253
3290000 384.53471304608235
3300000 376.2581588623184
3310000 378.375680146489
3320000 387.16838702711993
3330000 515.081133041247
3340000 450.52004387412956
3350000 355.9760617584546
3360000 357.0265927793983
3370000 362.69959121019576
3380000 394.7866128406764
3390000 357.7983192070128
3400000 370.22051271518296
3410000 358.68575503619616
3420000 373.7529015414116
3430000 380.4556524140518
3440000 357.49773197005226
3450000 363.975538284

6190000 365.1721363152544
6200000 350.9232533293641
6210000 367.96201195815087
6220000 372.07650049937587
6230000 352.47614640160197
6240000 373.5723565554725
6250000 366.003414019831
6260000 374.6611089554173
6270000 374.27389904631207
6280000 382.55153623906284
6290000 372.69542463452075
6300000 370.3803184661105
6310000 369.9699950585096
6320000 374.1025365968045
6330000 352.9914941573475
6340000 357.75266433032255
6350000 373.48326763629706
6360000 352.80782780710086
6370000 358.9166465094242
6380000 354.9897278634998
6390000 358.6996450258344
6400000 345.24879641521073
6410000 341.79197966226326
6420000 354.4202494741943
6430000 344.4285818151041
6440000 334.866322125114
6450000 360.92037428396134
6460000 341.22960276627936
6470000 359.63222689529823
6480000 343.7357052257097
6490000 351.41822543756496
6500000 334.02174669823364
6510000 372.4729139176588
6520000 336.5203719049955
6530000 367.9334765259976
6540000 358.94770111976
6550000 386.61919791804115
6560000 359.2169168532834

9310000 352.16255736277753
9320000 362.0458635033596
9330000 396.7247114608827
9340000 364.10558706159276
9350000 349.1482032899501
9360000 362.94676939935994
9370000 346.9393782332748
9380000 362.57299382524957
9390000 342.5448833568351
9400000 361.7922455964209
9410000 339.81456750103973
9420000 357.5561092076025
9430000 346.5556145882432
9440000 355.17248417359906
9450000 356.75033575761205
9460000 391.30813755005715
9470000 345.7696897322837
9480000 378.35385173293776
9490000 369.1411469765184
9500000 381.18624669787846
9510000 365.5092440691954
9520000 369.57793643871946
9530000 380.4335818967966
9540000 345.5511516143339
9550000 369.70623847992283
9560000 318.76200156586935
9570000 359.0492103929102
9580000 304.21251702133594
9590000 384.6455011336063
9600000 385.57929831473325
9610000 195.24480972806515
9620000 47.49525214222329
9630000 46.76964329462352
9640000 47.02356373483007
9650000 47.04193032810815
9660000 46.87072071003922
9670000 62.11898334877218
9680000 491.0805275949

12330000 347.6073388700045
12340000 341.3910558865416
12350000 365.65450100873306
12360000 357.0278793065143
12370000 358.77495866075316
12380000 347.78609567790113
12390000 357.34492009922644
12400000 361.41214450400935
12410000 365.8715467610919
12420000 345.5616439372828
12430000 379.0537255379016
12440000 352.8162324088302
12450000 361.53150367612614
12460000 351.63334058879775
12470000 332.1148666919663
12480000 352.91733947078905
12490000 342.32405421552915
12500000 368.01163405959403
12510000 320.6001889102589
12520000 363.0862421403569
12530000 299.34166511939054
12540000 339.8633199462996
12550000 322.50677730232064
12560000 408.4119707392936
12570000 284.6433433078046
12580000 46.05047817396621
12590000 46.32533654427892
12600000 46.14061230782129
12610000 45.82820559536021
12620000 46.68720885627208
12630000 46.50728542385018
12640000 347.0026319542941
12650000 432.81641184235343
12660000 322.62947983076515
12670000 337.3403878748489
12680000 295.32468035306965
12690000 346.

...

505140000 3.575950308660222
505150000 4.09727438727502
505160000 4.052533411177635
505170000 4.4683818000007
505180000 3.898574226166295
505190000 3.899801280795972
505200000 3.6060541081841455
505210000 1.845955126437684
505220000 3.0706821557795605
505230000 2.197800832309286
505240000 3.664967308763365
505250000 3.9448166637730595
505260000 3.790024696742786
505270000 4.363360193897664
505280000 4.103009652745426
505290000 3.982556431788968
505300000 4.233079008379989
505310000 1.1604216085322498
505320000 2.112819016113122
505330000 4.171370540129806
505340000 3.9042833537984496
505350000 4.101755542423486
505360000 4.539235984624094
505370000 4.070937829045038
505380000 4.217015756328504
505390000 3.1215315024510493
505400000 1.7630742839263875
505410000 2.1031879007903007
505420000 3.3818882277268836
505430000 4.409246473706544
505440000 3.8388653493513067
505450000 4.434533578582777
505460000 4.12563087955616
505470000 4.055005556155152
505480000 4.1353171703734395
505490000 3.0

507980000 1.5722820321291087
507990000 1.5925300888221396
508000000 1.652730640325308
508010000 1.5816893287135563
508020000 1.5428848629714649
508030000 1.4980181695317427
508040000 1.5397834917950035
508050000 1.4643794485496149
508060000 1.2282090451384517
508070000 1.009891327801744
508080000 0.6473624980036616
508090000 0.6978883998114931
508100000 0.640908847625196
508110000 0.697509252853963
508120000 1.3136547070553501
508130000 1.4764970426276922
508140000 1.460298526091496
508150000 1.570182097635329
508160000 1.484902537478268
508170000 1.4598881373601291
508180000 1.718172611054533
508190000 1.5758102271142402
508200000 1.4094042293871376
508210000 1.3691587026501297
508220000 1.4208396182694436
508230000 1.4546654937232997
508240000 1.5052274334322413
508250000 1.2173150053798159
508260000 0.6240227877536217
508270000 0.6883108341900376
508280000 0.6087188950523138
508290000 0.5935474682806995
508300000 1.0278452393561668
508310000 1.3735849092573524
508320000 1.3186428373

510750000 0.39541676985642643
510760000 0.3424471255247328
510770000 0.41240387819910046
510780000 0.7552833367204005
510790000 1.095680586615761
510800000 1.0029386221064727
510810000 1.2459031821922064
510820000 1.1126073150078788
510830000 1.0060106950141787
510840000 1.0150034444959626
510850000 0.9283419421930114
510860000 0.9991962176557381
510870000 0.9510381670855615
510880000 1.0276121683357424
510890000 1.0172905767072637
510900000 0.8781650023308264
510910000 0.9493404124798906
510920000 0.8323322031198144
510930000 0.90359882427703
510940000 0.9394900666711861
510950000 0.9144778281797568
510960000 0.885148398037725
510970000 0.8669250848183301
510980000 0.9203059094267287
510990000 0.9662424481951924
511000000 0.9027287530063457
511010000 0.8773014487423898
511020000 0.8020237332818839
511030000 0.756534706747426
511040000 0.8114588378622135
511050000 0.7515484344935947
511060000 0.8126567745605707
511070000 0.8625050910797714
511080000 0.9159116209244396
511090000 0.95105

513300000 0.8674872927087083
513310000 0.8850070716354914
513320000 1.3750649547395308
513330000 0.8502790541914305
513340000 0.9551000636137924
513350000 0.6295001695188285
513360000 0.5585106966169278
513370000 1.2424616337354448
513380000 1.1701496545517445
513390000 0.7924127894619637
513400000 0.4758235343105263
513410000 0.840730167509238
513420000 0.6593919748902055
513430000 1.1847436928255624
513440000 0.7022364518204928
513450000 0.8792464197808968
513460000 0.5007673646045194
513470000 0.8175602624230782
513480000 0.9397847991673681
513490000 0.7708920113547975
513500000 1.0100454351097943
513510000 0.3874113161926667
513520000 0.885225466989365
513530000 0.6387762062265276
513540000 1.088531324551026
513550000 0.8178740073446432
513560000 0.893908826938669
513570000 0.5244603103553983
513580000 1.1463424015125234
513590000 0.8012793158666491
513600000 0.7531602245047357
513610000 1.1169635371172957
513620000 0.49318247495466466
513630000 1.124640106100546
513640000 0.767741

515840000 0.020697846330821515
best so far: 0
type: [3, 2, 19, 1, 1] 114
cases of this type: 722
best so far: 0
type: [3, 19, 1, 1, 2] 114
cases of this type: 304
best so far: 0
type: [3, 19, 1, 2, 1] 114
cases of this type: 152
best so far: 0
type: [3, 19, 2, 1, 1] 114
cases of this type: 76
best so far: 0
type: [3, 38, 1, 1, 1] 114
cases of this type: 38
515850000 0.009643045814481046
best so far: 0
type: [6, 1, 1, 1, 19] 114
cases of this type: 130321
515860000 0.02136790488264958
515870000 0.017137724888026715
515880000 0.015100845314462978
515890000 0.01464154602884584
515900000 0.012988076740046341
515910000 0.006375794993003209
515920000 0.0049371084865927695
515930000 0.004658852839847406
515940000 0.007194358433670467
515950000 0.0063010634646879305
515960000 0.0048802668380737305
515970000 0.002913869162950251
515980000 0.0017324012530446052
best so far: 0
type: [6, 1, 1, 19, 1] 114
cases of this type: 6859
best so far: 0
type: [6, 1, 19, 1, 1] 114
cases of this type: 361
bes